# k8s环境搭建

## 目标

搭建一个多master节点、多worker节点集群环境，用于学习kubernets的架构（公司的kubernetes环境受限，无法使用k8s新特性），并将自己的应用服务迁移部署到k8s集群内。

## 环境准备

etcd集群至少需要3个节点，因此master高可用集群也至少需要3个节点，另外配置2个工作节点。

华为云等云服务提供商租赁虚拟机VM的成本比较高，因此在物理上通过VirtualBox虚拟化技术虚拟VM。

### VirtualBox构建虚拟VM

物理机软、硬件环境：

|资源|配置|
|---|---|
|CPU|20 Core|
|内存|64GB|
|磁盘|SSD 1T、HHD 2T|
|操作系统|Ubuntu 18.04|

网络要求：

* 虚拟机之间能互相通信
* 虚拟机能访问公网

**开始使用NAT Network网络模式，但过段时间后物理机和虚拟机之间无法进行网络通信，后面将网络模式切换为桥接模式。**

软件版本：

|软件|版本|
|---|---|
|VirtualBox|`6.0.24`|
|镜像|`Ubuntu 18.04`|

> **VirtualBox尽量匹配上述版本。**

实操过程省略。

### 配置虚拟机免密登录

机器列表：

`/etc/hosts`

```
# vms
172.30.12.169 vm1
172.30.12.170 vm2
172.30.12.171 vm3
172.30.12.172 vm4
172.30.12.175 vm5
```

VM1作为主要的ssh登录机器，能通过ssh免密登录到其他所有机器。

1. VM安装`openssh-server`
2. 配置VM1免密登录其他所有VM

实操过程省略。

### 配置虚拟机sudo免密输入

更新`/etc/sudoers`文件

sudo vim `/etc/sudoers`

```
user_name ALL=(ALL:ALL) ALL
user_name ALL=(ALL) NOPASSWD: ALL
```

### 配置物理机登录虚拟机

虚拟机到物理机的ssh隧道：

1. 虚拟机安装autossh `sudo apt install autossh`，创建ssh隧道 `autossh -NfR 9999:localhost:22 user_name@ip`
2. 物理机安装sshpass `sudo apt install sshpass`，登录到虚拟机 `sshpass -p user_name ssh localhost -p9999`

> 网络模式从**NAT Network**改成**桥接**模式后，4层网络是互通的，不再需要端口或ssh隧道。

### 更新Ubuntu软件源

参考清华大学开源软件镜像站: https://mirrors.tuna.tsinghua.edu.cn/help/ubuntu/

初始化系统环境：

```shell
sudo apt-get -y install build-essential libssl-dev git zsh

```

### 安装ohmyzsh

参考文献：https://github.com/ohmyzsh/ohmyzsh

## 搭建k8s集群

### 环境准备

参考文献：

* https://v1-19.docs.kubernetes.io/zh/docs/setup/production-environment/tools/kubeadm/install-kubeadm/

#### 配置Linux

```shell

# 禁用交换分区
sudo swapoff -a
sudo sed -i '/swap/s/^/#/' /etc/fstab

# 配置 br_netfilter
cat <<EOF | sudo tee /etc/sysctl.d/k8s.conf
net.bridge.bridge-nf-call-ip6tables=1
net.bridge.bridge-nf-call-iptables=1
EOF

sudo sysctl --system

# 禁用 SELinux，Debian默认没有安装Selinux，CentOS默认安selinux
# sudo setenforce 0
# sudo sed -i 's/^SELINUX=enforcing$/SELINUX=disabled/' /etc/selinux/config

# 安装ntp
sudo apt install -y ntpdate

# 关闭防火墙
sudo ufw disable
```

#### 安装docker

参考文献：

* https://phoenixnap.com/kb/how-to-install-docker-on-ubuntu-18-04
* *docker notebook*

#### 安转kubeadm

```shell
sudo apt update && sudo apt install -y apt-transport-https curl
curl -s https://mirrors.aliyun.com/kubernetes/apt/doc/apt-key.gpg | sudo apt-key add -

cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list
deb https://mirrors.aliyun.com/kubernetes/apt/ kubernetes-xenial main
EOF

# 安装kubeadm
sudo apt update 
sudo apt install -y kubeadm=1.19.16-00  kubelet=1.19.16-00 kubectl=1.19.16-00
```

> `apt-cache madison kubeadm`查看历史版本

### 创建集群

参考文献：

* [使用 kubeadm 创建集群](https://v1-19.docs.kubernetes.io/zh/docs/setup/production-environment/tools/kubeadm/create-cluster-kubeadm/)
* [利用 kubeadm 创建高可用集群](https://kubernetes.io/zh/docs/setup/production-environment/tools/kubeadm/high-availability/)
* [搭建高可用负载均衡器: haproxy+keepalived](https://developer.aliyun.com/article/609851)
* [High Availability Considerations](https://github.com/kubernetes/kubeadm/blob/main/docs/ha-considerations.md#options-for-software-load-balancing)

#### 搭建高可用负载均衡器

```shell
#安装keepalived haproxy
sudo apt install -y keepalived haproxy

# 配置keepalived、haproxy

# 重启keepalived、haproxy
sudo systemctl restart keepalived
sudo systemctl restart haproxy
```

keepalvied、haproxy配置参考：[High Availability Considerations](https://github.com/kubernetes/kubeadm/blob/main/docs/ha-considerations.md#options-for-software-load-balancing)

>VIP为`172.30.12.250`，端口为`6443`

#### 初始化控制平面

```shell
sudo kubeadm init \
        --apiserver-advertise-address 0.0.0.0 \
        --apiserver-bind-port 6443 \
        --control-plane-endpoint "172.30.12.250:8443" \
        --upload-certs \
        --image-repository registry.cn-hangzhou.aliyuncs.com/google_containers 
```

> 通过`--image-repository`更换默认的镜像仓库地址。

首次初始化，终端最后输出日志：

```log
To start using your cluster, you need to run the following as a regular user:

  mkdir -p $HOME/.kube
  sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config
  sudo chown $(id -u):$(id -g) $HOME/.kube/config

You should now deploy a pod network to the cluster.
Run "kubectl apply -f [podnetwork].yaml" with one of the options listed at:
  https://kubernetes.io/docs/concepts/cluster-administration/addons/

You can now join any number of the control-plane node running the following command on each as root:

  kubeadm join 172.30.12.250:8443 --token 07x0ps.84245sbldawrta77 \
    --discovery-token-ca-cert-hash sha256:836e505073fde384891dff86386f499b2ed1146f141326c8c02c145aa4a17c1f \
    --control-plane --certificate-key 7b45c0656c53b5fde7b46cb2c50ed46e0088aec213061e336aa8c722a1aad021

Please note that the certificate-key gives access to cluster sensitive data, keep it secret!
As a safeguard, uploaded-certs will be deleted in two hours; If necessary, you can use
"kubeadm init phase upload-certs --upload-certs" to reload certs afterward.

Then you can join any number of worker nodes by running the following on each as root:

kubeadm join 172.30.12.250:8443 --token 07x0ps.84245sbldawrta77 \
    --discovery-token-ca-cert-hash sha256:836e505073fde384891dff86386f499b2ed1146f141326c8c02c145aa4a17c1f
```

配置kubectl访问k8s集群：

```shell
  mkdir -p $HOME/.kube
  sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config
  sudo chown $(id -u):$(id -g) $HOME/.kube/config
```

#### 安装网络插件

```shell
wget https://docs.projectcalico.org/v3.8/manifests/calico.yaml
kubectl apply -f calico.yaml
```

#### 加入其他master节点

先初始化环境，安装kubeadm、kubelet和kubectl，再执行`kubeadm join`

```shell
sudo kubeadm join 172.30.12.250:8443 --token 07x0ps.84245sbldawrta77 \
    --discovery-token-ca-cert-hash sha256:836e505073fde384891dff86386f499b2ed1146f141326c8c02c145aa4a17c1f \
    --control-plane --certificate-key 7b45c0656c53b5fde7b46cb2c50ed46e0088aec213061e336aa8c722a1aad021
```

#### 加入工作节点

先初始化环境，安装kubeadm、kubelet和kubectl，再执行`kubeadm join`

```shell
sudo kubeadm join 172.30.12.250:8443 --token 07x0ps.84245sbldawrta77 \
    --discovery-token-ca-cert-hash sha256:836e505073fde384891dff86386f499b2ed1146f141326c8c02c145aa4a17c1f
```

### 集成metrics-server

参考文献：*k8s集成metrics-server notebook*

### 配置默认存储类

参考文献：*k8s存储类 notebook*

### 集成kubesphere

参考文献：*k8s集成kubesphere*